# User Interface

> comming soon...

In [ ]:
#| hide
import os
import sys
sys.path.append(os.path.dirname(os.getcwd())) # To add the parent directory to the path
from scholaris.core import *

In [ ]:
#| default_exp ui